In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir("..")  # Move up one level to the project root

In [ ]:
import pandas as pd
import plotly.express as px
from src.time_series.preprocessing import PrePro
from src.time_series.validation import Validation
from src.time_series.analysis import Analysis
from statsmodels.tsa.stattools import acf
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import STL
import statsmodels.api as sm
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
print(os.getcwd())

In [ ]:
#"/Users/omare/Desktop/personal_project/time-series/data/DailyDelhiClimateTrain.csv"

file_path_train = "./data/DailyDelhiClimateTrain.csv"
file_path_test = "./data/DailyDelhiClimateTest.csv"

In [ ]:
train_data = pd.read_csv(file_path_train)
test_data = pd.read_csv(file_path_test)

In [ ]:
prepro = PrePro()
validation = Validation()
analysis = Analysis()

In [ ]:
prepro.to_datetime(df=train_data, name_column="date")
prepro.to_datetime(df=test_data, name_column="date")

In [ ]:
train_data.info()

In [ ]:
validation.plot_time_series(df=train_data, x="date", y="meantemp", title=f"time series temperature")

In [ ]:
rolling_window_plot, adfuller_test=analysis.check_stationarity(train_data, name="meantemp", window_size=12)

In [ ]:
analysis.acf_plot(train_data, "meantemp", 3000)

In [ ]:
# I take the seasonality as the difference in lag between two peaks
fig, result = analysis.decomposition_time_series(train_data, "meantemp", 362, "Mean Temp" )

In [ ]:
fig

In [ ]:
residuals = pd.DataFrame({"meantemp":result.resid, "date_datetime":train_data["date_datetime"]})
trend = pd.DataFrame({"meantemp":result.trend, "date_datetime":train_data["date_datetime"]})
seasonality = pd.DataFrame({"meantemp":result.seasonal, "date_datetime":train_data["date_datetime"]})

In [ ]:
validation.plot_time_series(df=residuals, x="date_datetime", y="meantemp", title=f"time series temperature")

In [ ]:
rolling_window_plot, adfuller_test=analysis.check_stationarity(residuals, name="meantemp", window_size=12)

In [ ]:
rolling_window_plot

In [ ]:
adfuller_test

In [ ]:
analysis.acf_plot(residuals, "meantemp", 40)
# Mi sembra che sia autoregressiva. Non so se c'è anche una componente di mooving average.

In [ ]:
analysis.pacf_plot(residuals, "meantemp", 40)

In [ ]:
forecast_residuals = analysis.forecast_component(residuals,test_data, (1,0,1),)
forecast_trend = analysis.forecast_component(trend, test_data, (1,4,1))
forecast_residuals.reset_index(inplace=True)
forecast_trend.reset_index(inplace=True)

In [ ]:
start_date = test_data["date_datetime"].iloc[0]
end_date = test_data["date_datetime"].iloc[-1]
print(start_date, end_date)


In [ ]:
start_date_seasonal = "2013-01-01"
end_date_seasonal = "2013-04-24"
start_date_seasonal = pd.to_datetime(start_date_seasonal)
end_date_seasonal = pd.to_datetime(end_date_seasonal)

forecast_seasonal = seasonality[(seasonality['date_datetime'] >= start_date_seasonal) & (seasonality['date_datetime'] <= end_date_seasonal)]

# Display the resulting DataFrame
forecast_seasonal.reset_index(inplace=True)

In [ ]:
forecast = pd.DataFrame()
forecast["meantemp"] = forecast_trend["forecast"] + forecast_seasonal["meantemp"]+ forecast_residuals["forecast"]
forecast["date_datetime"] = forecast_trend["date_datetime"]

In [ ]:
mse = validation.mse_error(forecast, test_data, "meantemp" )
print(f"Mean squared error: {round(mse,2)} %")

In [ ]:
validation.evaluate_forecast_plot(forecast, test_data, "meantemp", "Mean Temperature" )